In [1]:
# !pip install tensorflow
!pip install openpyxl

In [2]:
import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta
import os
import glob
import s3fs
import boto3
import random
from config_module import Config
config = Config()

os.environ['PYTHONHASHSEED'] = str(2)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

from sklearn.preprocessing import MinMaxScaler
# import torch
# import tensorflow as tf
# import tensorflow.keras as keras
# from tensorflow.keras.models import save_model, model_from_json, Sequential
# from tensorflow.keras.layers import Dense, LSTM, Dropout
from pickle import dump, load
# from keras.models import load_model
import warnings
# from torch import nn
# import torch.nn.functional as F

import random
# def reset_random_seeds():
#     os.environ['PYTHONHASHSEED'] = str(2)
#     tf.random.set_seed(2)
#     np.random.seed(2)
#     random.seed(2)
#     tf.keras.utils.set_random_seed(2024)

# # Make some random data
# reset_random_seeds()

# Set seed for reproducibility
np.random.seed(42)
# tf.random.set_seed(42)
pd.options.mode.chained_assignment = None

# Import modules
import itertools
import logging
import math
import pickle
from datetime import datetime

import ast
import seaborn as sn
#import xgboost as xgb
from matplotlib import pyplot as plt
from pandas.plotting import register_matplotlib_converters
#from prophet import Prophet
from sklearn.model_selection import ParameterGrid
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.metrics import (mean_absolute_error, mean_absolute_percentage_error,
                             mean_squared_error)
from sklearn.model_selection import (GridSearchCV, TimeSeriesSplit,
                                     train_test_split)
from sklearn.preprocessing import StandardScaler
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm
warnings.filterwarnings('ignore')
random.seed(12)

In [3]:
def read_file(file_path):
    file_extension = os.path.splitext(file_path)[1].lower()
    
    if file_extension == '.xlsx':
        # Read Excel file
        data = pd.read_excel(file_path)
    elif file_extension == '.csv':
        # Read CSV file
        data = pd.read_csv(file_path)
    elif file_extension == '.txt':
        # Read TXT file
        data = pd.read_csv(file_path)
    else:
        raise ValueError("Unsupported file format: {}".format(file_extension))
    
    return data

def get_latest_file(folder_path, file_prefix):
    # Construct the search pattern for the CSV files with the given prefix
    search_pattern = os.path.join(folder_path, f"{file_prefix}*.csv")
    
    # Get the list of matching files
    csv_files = glob.glob(search_pattern)
    
    if not csv_files:
        print("No CSV files found.")
        return None
    
    # Extract the datetime part and convert it to a datetime object
    csv_files_with_dates = []
    for file in csv_files:
        try:
            # Extract the datetime stamp from the file name
            datetime_stamp = file[len(folder_path) + len(file_prefix) + 1:-5]
            file_datetime = datetime.strptime(datetime_stamp, "%y%m%d_%H%M%S")
            csv_files_with_dates.append((file, file_datetime))
        except ValueError:
            print(f"Could not parse date from file name: {file}")
    
    if not csv_files_with_dates:
        print("No valid CSV files found.")
        return None
    
    # Sort the files by datetime in descending order and get the latest one
    latest_file = max(csv_files_with_dates, key=lambda x: x[1])[0]
    
    return latest_file

In [4]:
def get_consecutive_months(start_date, n_months=3):
    """
    Generate a list of consecutive month start dates.

    Args:
    - start_date (str): Starting date in 'YYYY-MM-DD' format.
    - n_months (int): Number of consecutive months to generate.

    Returns:
    - list: List of consecutive month start dates in 'YYYY-MM-DD' format.
    """
    start_date = pd.to_datetime(start_date)
    month_list = [(start_date + pd.DateOffset(months=i)).strftime('%Y-%m-%d') for i in range(1, n_months + 1)]
    return month_list

In [5]:
def read_file_from_name(output_path_url, model_filename_preffix):
    model_file_path = get_latest_file(output_path_url, model_filename_preffix)
    print(model_file_path)
    model_forecast_data = read_file(model_file_path)
    return model_forecast_data

In [6]:
fs = s3fs.S3FileSystem()
bucket = config.get_value('bucket')
#prefix = config.get_value('prefix')
prefix_inp = config.get_value('prefix_inp')
inp_path_forecast = config.get_value('inp_path_forecast')

In [7]:
# part_master_path_1 = f"s3://s3-dx-tdem-ie-nonprod-persist/persist-bo/persist-bo-demand-forecast/to-be-removed/test_data/TPCAP Part Master New.xlsx"
# data_gt_master_path_1=f"s3://s3-dx-tdem-ie-nonprod-persist/persist-bo/persist-bo-demand-forecast/to-be-removed/master_data/job3/etljob3input/GT_part_master.csv"
# data_pic_master_path_1=f"s3://s3-dx-tdem-ie-nonprod-persist/persist-bo/persist-bo-demand-forecast/to-be-removed/master_data/job3/etljob3input/PIC_master.csv"
# data_supp_master_path_1=f"s3://s3-dx-tdem-ie-nonprod-persist/persist-bo/persist-bo-demand-forecast/to-be-removed/master_data/job3/etljob3input/Supplier_master.csv"

In [8]:
print(inp_path_forecast)

persist-bo/persist-bo-demand-forecast/model-forecasted-output/


In [9]:
def get_latest_s3_file(bucket_name, path_prefix, file_prefix):
    s3 = boto3.client('s3')
    
    # List all objects under the given path_prefix
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=path_prefix)
    
    if 'Contents' not in response:
        print("No files found.")
        return None
    
    # Filter objects that start with the specific file path_prefix
    filtered_files = [
        obj for obj in response['Contents'] 
        if obj['Key'].startswith(f"{path_prefix}{file_prefix}")
    ]
    
    if not filtered_files:
        print("No files found with the specific path_prefix.")
        return None
    
    # Extract the date from each file name and store along with the key
    files_with_dates = []
    for obj in filtered_files:
        key = obj['Key']
        # Assuming the date is at the end of the filename after the last underscore
        try:
            # Get the base name without directory path
            base_name = os.path.basename(key)
            name_without_extension = os.path.splitext(base_name)[0]
            date_str = name_without_extension.split('_')[-1]  # Get the date string part
            file_date = datetime.strptime(date_str, "%Y-%m-%d")  # Parse date
            files_with_dates.append((key, file_date))
        except ValueError as e:
            print(f"Error parsing date from file name {key}: {e}")

    if not files_with_dates:
        print("No valid files with dates found.")
        return None

    # Find the file with the latest date
    latest_file_key, latest_date = max(files_with_dates, key=lambda x: x[1])
    
    return latest_file_key

In [10]:
inp_path_forecast+f"{config.get_value('model_forecasting_date')}/"

'persist-bo/persist-bo-demand-forecast/model-forecasted-output/20241103/'

In [11]:
prophet_forecast_file_name = config.get_value('prophet_forecast_file_name')
prophet_forecast_data_path = get_latest_s3_file(bucket, inp_path_forecast+f"{config.get_value('model_forecasting_date')}/", prophet_forecast_file_name)
prophet_forecast_data_1 = pd.read_excel('s3://'+f'{bucket}/{prophet_forecast_data_path}')


lstm_forecast_file_name = config.get_value('lstm_forecast_file_name')
lstm_forecast_data_path = get_latest_s3_file(bucket, inp_path_forecast+f"{config.get_value('model_forecasting_date')}/", lstm_forecast_file_name)
lstm_forecast_data_1 = pd.read_excel('s3://'+f'{bucket}/{lstm_forecast_data_path}')

lasso_forecast_file_name = config.get_value('lasso_forecast_file_name')
lasso_forecast_data_path = get_latest_s3_file(bucket, inp_path_forecast+f"{config.get_value('model_forecasting_date')}/", lasso_forecast_file_name)
lasso_forecast_data_1 = pd.read_excel('s3://'+f'{bucket}/{lasso_forecast_data_path}')

ridge_forecast_file_name = config.get_value('ridge_forecast_file_name')
ridge_forecast_data_path = get_latest_s3_file(bucket, inp_path_forecast+f"{config.get_value('model_forecasting_date')}/", ridge_forecast_file_name)
ridge_forecast_data_1 = pd.read_excel('s3://'+f'{bucket}/{ridge_forecast_data_path}')

xgb_forecast_file_name = config.get_value('xgb_forecast_file_name')
xgb_forecast_data_path = get_latest_s3_file(bucket, inp_path_forecast+f"{config.get_value('model_forecasting_date')}/", xgb_forecast_file_name)
xgb_forecast_data_1 = pd.read_excel('s3://'+f'{bucket}/{xgb_forecast_data_path}')

last_train_date = (pd.to_datetime(prophet_forecast_data_1['period']).max() + relativedelta(months=-3)).strftime('%Y-%m-%d')

In [12]:
print(prophet_forecast_data_path)
print(xgb_forecast_data_path)
print(lstm_forecast_data_path)
print(ridge_forecast_data_path)
print(lasso_forecast_data_path)

persist-bo/persist-bo-demand-forecast/model-forecasted-output/20241103/prophet_forecast_data_2024-11-03.xlsx
persist-bo/persist-bo-demand-forecast/model-forecasted-output/20241103/xgb_forecast_data_2024-11-03.xlsx
persist-bo/persist-bo-demand-forecast/model-forecasted-output/20241103/lstm_forecast_data_2024-11-03.xlsx
persist-bo/persist-bo-demand-forecast/model-forecasted-output/20241103/ridge_forecast_data_2024-11-03.xlsx
persist-bo/persist-bo-demand-forecast/model-forecasted-output/20241103/lasso_forecast_data_2024-11-03.xlsx


In [13]:
xgb_forecast_data_1['part_number'].nunique(), prophet_forecast_data_1['part_number'].nunique(), lstm_forecast_data_1['part_number'].nunique(), lasso_forecast_data_1['part_number'].nunique(), ridge_forecast_data_1['part_number'].nunique()

(1101, 3741, 155, 795, 733)

In [14]:
xgb_forecast_data_1['part_number'].nunique(), prophet_forecast_data_1['part_number'].nunique(), lstm_forecast_data_1['part_number'].nunique(), lasso_forecast_data_1['part_number'].nunique(), ridge_forecast_data_1['part_number'].nunique()

(1101, 3741, 155, 795, 733)

In [15]:
def normalize_forecast(df_forecast, colname):
    df_forecast[colname] = df_forecast[colname].apply(lambda x:int(round(x)))
    df_forecast[colname] = df_forecast[colname].apply(lambda x: np.where(x<0,max(x,0),x))
    return df_forecast

In [16]:
def forecast_data_merge(prophet_forecast_data,lstm_forecast_data,lasso_forecast_data,ridge_forecast_data,xgb_forecast_data,
                        last_training_date, fitted_data):
    
    forecasted_data_prophet = prophet_forecast_data[prophet_forecast_data['period'].isin(get_consecutive_months(last_training_date))].reset_index(drop=True)
    forecasted_data_lstm = lstm_forecast_data[lstm_forecast_data['period'].isin(get_consecutive_months(last_training_date))].reset_index(drop=True)
    forecasted_data_lasso = lasso_forecast_data[lasso_forecast_data['period'].isin(get_consecutive_months(last_training_date))].reset_index(drop=True)
    forecasted_data_ridge = ridge_forecast_data[ridge_forecast_data['period'].isin(get_consecutive_months(last_training_date))].reset_index(drop=True)
    forecasted_data_xgb = xgb_forecast_data[xgb_forecast_data['period'].isin(get_consecutive_months(last_training_date))].reset_index(drop=True)
    
    forecasted_data_prophet_subset = forecasted_data_prophet.drop(['actual', 'type', 'model_name'], axis=1).rename(columns={'fitted':'forecasted'})
    forecasted_data_prophet_subset['model_name']='Prophet'
    forecasted_data_lstm_subset = forecasted_data_lstm.drop(['Actual', 'last_history'], axis=1).rename(columns={'Forecast':'forecasted'})
    forecasted_data_lstm_subset['model_name'] = 'LSTM single model'
    forecasted_data_lasso_subset = forecasted_data_lasso.drop(['Actual_values'], axis=1).rename(columns={'Predicted_values_lag1':'forecasted'})
    forecasted_data_lasso_subset['model_name'] = 'Lasso'
    forecasted_data_ridge_subset = forecasted_data_ridge.drop(['Actual_values'], axis=1).rename(columns={'Predicted_values_lag1':'forecasted'})
    forecasted_data_ridge_subset['model_name'] = 'Ridge'
    forecasted_data_xgb_subset = forecasted_data_xgb.drop(['Actual_values'], axis=1).rename(columns={'Predicted_values_lag1':'forecasted'})
    forecasted_data_xgb_subset['model_name'] = 'XGBoost'

    forecasted_data_prophet_subset = forecasted_data_prophet_subset[forecasted_data_lstm_subset.columns]
    forecasted_data_ridge_subset = forecasted_data_ridge_subset[forecasted_data_lstm_subset.columns]
    forecasted_data_xgb_subset = forecasted_data_xgb_subset[forecasted_data_lstm_subset.columns]
    forecasted_data_lasso_subset = forecasted_data_lasso_subset[forecasted_data_lstm_subset.columns]

    # Concating all forecasts
    final_forecasted_data = pd.concat([forecasted_data_prophet_subset, 
                                       forecasted_data_lstm_subset, 
                                       forecasted_data_ridge_subset, 
                                       forecasted_data_xgb_subset, 
                                       forecasted_data_lasso_subset], ignore_index=True)
    
#     final_forecasted_data = normalize_forecast(final_forecasted_data, 'forecasted')
    final_forecasted_data['period'] = pd.to_datetime(final_forecasted_data['period']).astype(str)
    pivot_df = final_forecasted_data.pivot_table(index=['part_number', 'model_name'], 
                                                  columns='period', 
                                                  values=['forecasted'], 
                                                  sort=False)
    # Flatten the column names
    pivot_df.columns = [f'{col[1]}_{col[0]}' for col in pivot_df.columns]
    
    # Sort the columns
    pivot_df = pivot_df.sort_index(axis=1).reset_index()
    
#     part_master = pd.read_excel(part_master_path)
#     part_master['ICC_seg'] = part_master['ICC'].apply(lambda x:x[0])
    
    # Extract the static columns and the dynamic date columns
    static_columns = ['part_number', 'model_name']
    date_columns = [col for col in pivot_df.columns if col not in static_columns]

    # Function to extract date and type
    def extract_date_and_type(col_name):
        parts = col_name.split('_')
        date = parts[0]
        col_type = '_'.join(parts[1:])
        return date, col_type

    # Sort the date columns based on date and type
    sorted_date_columns = sorted(date_columns, key=lambda x: (extract_date_and_type(x)[0], ['forecasted'].index(extract_date_and_type(x)[1])))

    # Combine static columns with sorted date columns
    sorted_columns = static_columns + sorted_date_columns

    # Reorder the DataFrame columns
    final_data_compared_bestfit = pivot_df[sorted_columns]
    final_data_compared_bestfit['Confidence_interval'] = '#N/A'
    final_data_compared_bestfit.loc[final_data_compared_bestfit['model_name']=="Prophet", 'Confidence_interval'] = 0.8
    
    fitted_data_threshold = fitted_data[['Part Number', 'Threshold']].drop_duplicates().reset_index(drop=True)
    final_data_compared_bestfit_merged = final_data_compared_bestfit.merge(fitted_data_threshold, left_on='part_number', 
                                                                             right_on='Part Number', how='left')
    df_melted = pd.melt(final_data_compared_bestfit_merged, id_vars=['part_number', 'model_name', 'Confidence_interval','Threshold'],
                        value_vars=date_columns, 
                        var_name='date_measure', 
                        value_name='value')

    # Extract date and measure type
    df_melted[['date', 'measure']] = df_melted['date_measure'].str.extract(r'(\d{4}-\d{2}-\d{2})_(\w+)')
    df_melted.drop('date_measure', axis=1, inplace=True)

    # Pivot the DataFrame
    df_pivoted = df_melted.pivot_table(index=['part_number', 'model_name', 'date'], 
                                       columns='measure', 
                                       values='value', 
                                       aggfunc='first').reset_index()

    # Merge with static columns
    static_cols = final_data_compared_bestfit_merged.drop(columns=date_columns).drop_duplicates(subset=['part_number', 'model_name'])
    result = pd.merge(df_pivoted, static_cols, on=['part_number', 'model_name'])

    # Rename columns for clarity
    result.columns.name = None
    result['actual'] = np.NaN
    result['accuracy_pred'] = np.NaN
    result = result.rename(columns={
        'actual': 'Actual',
        'forecasted': 'Forecasted',
        'accuracy_pred': 'Accuracy_Pred'
    })

    result_subset = result[['part_number', 'model_name', 'date', 'Actual', 'Forecasted', 
                            'Accuracy_Pred', 'Confidence_interval', 'Threshold']]
    return result_subset

In [17]:
# prefix_bestfit_file+"/"
# bestfit_powerbi_path
# print('s3://'+f'{bucket_new}/{bestfit_powerbi_path}')

In [18]:
bucket_new = config.get_value('bucket_new')
best_file_base_name_powerbi = config.get_value('best_file_base_name_powerbi')
prefix_bestfit_file = config.get_value('prefix_bestfit_file')
# prefix_bestfit_file = prefix_bestfit_file+'/'
bestfit_powerbi_path = get_latest_s3_file(bucket_new, prefix_bestfit_file+"/", best_file_base_name_powerbi)
print(1)
print(bestfit_powerbi_path)
fitted_data_1 = pd.read_excel('s3://'+f'{bucket_new}/{bestfit_powerbi_path}')

prefix_forecast_file = config.get_value('prefix_forecast_file')
file_name_bestfit_forecast = f'{config.get_value("file_name_bestfit_forecast")}{datetime.now().strftime("%Y-%m-%d")}.xlsx'
file_path_bestfit_forecast = f's3://{bucket_new}/{prefix_forecast_file}{file_name_bestfit_forecast}'

final_data_compared_bestfit_1 = forecast_data_merge(prophet_forecast_data_1,lstm_forecast_data_1,lasso_forecast_data_1,
                                ridge_forecast_data_1,xgb_forecast_data_1,last_train_date,fitted_data_1)
final_data_compared_bestfit_1.to_excel(file_path_bestfit_forecast, index=False)

1
curated-bo/curated-bo-demand-forecast/bestfit-selected-models/bestfit_selected_models_historic_months_powerbi_2024-11-03.xlsx


In [19]:
file_path_bestfit_forecast

's3://s3-dx-tdem-ie-nonprod-curated/curated-bo/curated-bo-demand-forecast/selected-models-forecasted-output/forecast_selected_models_future_months_2024-11-05.xlsx'

In [20]:
final_data_compared_bestfit_1

,part_number,model_name,date,Actual,Forecasted,Accuracy_Pred,Confidence_interval,Threshold
0,04000081B0,Prophet,2024-07-01,NaN,5.010885,NaN,0.8,True
1,04000081B0,Prophet,2024-08-01,NaN,7.912933,NaN,0.8,True
2,04000081B0,Prophet,2024-09-01,NaN,8.558024,NaN,0.8,True
3,0411106101,Prophet,2024-07-01,NaN,16.247474,NaN,0.8,True
4,0411106101,Prophet,2024-08-01,NaN,9.626274,NaN,0.8,True
...,...,...,...,...,...,...,...,...
19570,PZ08100005,Ridge,2024-08-01,NaN,158.944408,NaN,#N/A,True
19571,PZ08100005,Ridge,2024-09-01,NaN,160.446001,NaN,#N/A,True
19572,PZT0038023,Prophet,2024-07-01,NaN,324.423829,NaN,0.8,True
19573,PZT0038023,Prophet,2024-08-01,NaN,310.349778,NaN,0.8,True


#### power BI process

In [21]:
bucket_curated = config.get_value('bucket_new')
prefix_gtmaster = config.get_value('prefix_gtmaster')
prefix_proc = config.get_value('prefix_proc')
prefix_pic_master_path = config.get_value("prefix_pic_master_path")
product_master_path_prefix = config.get_value('product_master_path_prefix')

In [22]:
# Function to read a single Parquet file
def read_parquet_file(file):
    return pd.read_parquet(f's3://{file}', engine='pyarrow')

def gt_parquet_read(bucket_curated_name, gt_master_parquet):
    # Recursively list all Parquet files in the nested S3 path
#     parquet_files_gt_partm = fs.glob(f's3://{bucket_curated_name}/{gt_master_parquet}/*/*/*/*.parquet')
    parquet_files_gt_partm = fs.glob(f's3://{bucket_curated_name}/{gt_master_parquet}*.parquet')
    df_master_gt_part_master = pd.DataFrame()
    for file in parquet_files_gt_partm:
        df_gt = pd.read_parquet(f's3://{file}', engine='pyarrow')
        df_master_gt_part_master = pd.concat([df_master_gt_part_master,df_gt],ignore_index=True)
    df_master_gt_part_master_subset = df_master_gt_part_master.sort_values('sales_start_dt', ascending=False)
    df_master_gt_part_master_subset = df_master_gt_part_master_subset.drop_duplicates(subset=['part_no'], keep='first')
    df_master_gt_part_master_subset = df_master_gt_part_master_subset[["part_no", "country_origin_cd"]].drop_duplicates()
    df_master_gt_part_master_subset = df_master_gt_part_master_subset.assign(IMPORT_CD = 
                                                                             lambda x: np.where(x['country_origin_cd']=='TH', 
                                                                                                'LSP', 
                                                                                                np.where(x['country_origin_cd']=='JP', 
                                                                                                         'JSP', 'MSP')))
    df_master_gt_part_master_subset = df_master_gt_part_master_subset.drop('country_origin_cd', axis=1)
    df_master_gt_part_master_subset.columns = [col.upper() for col in df_master_gt_part_master_subset.columns]
    return df_master_gt_part_master_subset

def proc_parquet_read(bucket_curated_name, proc_master_parquet):
    # Recursively list all Parquet files in the nested S3 path
#     parquet_files_proc = fs.glob(f's3://{bucket_curated_name}/{proc_master_parquet}/*/*/*/*.parquet')
    parquet_files_proc = fs.glob(f's3://{bucket_curated_name}/{proc_master_parquet}*.parquet')

    df_master_proc_master = pd.DataFrame()

    for file in parquet_files_proc:
    #     print(file)
        df_proc = pd.read_parquet(f's3://{file}', engine='pyarrow')
        df_master_proc_master = pd.concat([df_master_proc_master,df_proc],ignore_index=True)
    df_master_proc_master = df_master_proc_master.sort_values(['start_dt'], ascending=[False])
    df_master_proc_master = df_master_proc_master.drop_duplicates(subset='part_no', keep='first')
    df_master_proc_master_subset = df_master_proc_master[['part_no', 'procurement_div_cd', 'procurement_operator_id', 'inventory_ctrl_class']].drop_duplicates()
    df_master_proc_master_subset['procurement_div_cd'] = df_master_proc_master_subset['procurement_div_cd'].apply(lambda x:x[-3:])
    df_master_proc_master_subset = df_master_proc_master_subset.rename(columns={'procurement_div_cd':'Vendor Code',
                                                                                'part_no':'PART_NO',
                                                                                'procurement_operator_id':'PROCUREMENT_OPERATOR_ID',
                                                                                'inventory_ctrl_class': 'ICC_seg'
                                                                       })
    df_master_proc_master_subset['ICC_seg'] = df_master_proc_master_subset['ICC_seg'].apply(lambda x: x[0] if pd.notna(x) and len(x) > 1 else x)
    return df_master_proc_master_subset

def pic_parquet_read(bucket_curated_name, pic_master_parquet):
    # Recursively list all Parquet files in the nested S3 path
#     parquet_files_pic_master = fs.glob(f's3://{bucket_curated_name}/{pic_master_parquet}/*/*/*/*.parquet')
    parquet_files_pic_master = fs.glob(f's3://{bucket_curated_name}/{pic_master_parquet}*.parquet')
    

    df_master_pic = pd.DataFrame()

    for file in parquet_files_pic_master:
    #     print(file)
        df_pic = pd.read_parquet(f's3://{file}', engine='pyarrow')
        df_master_pic = pd.concat([df_master_pic,df_pic],ignore_index=True)
    
    df_master_pic_subset = df_master_pic[['id', 'pic', 'pic_abb']].drop_duplicates().reset_index(drop=True)
    df_master_pic_subset.columns = [col.upper() for col in df_master_pic_subset.columns]

    return df_master_pic_subset

def prod_master_read(bucket_curated_name, pg_master_path):
    pg_master_filepath = "s3://"+fs.glob(f's3://{bucket_curated_name}/{pg_master_path}/*.csv')[0]
    pg_master_df = read_file(pg_master_filepath)
    pg_master_df = pg_master_df[['PART_NO', 'PRODUCT_GROUP']].drop_duplicates()
    return pg_master_df

In [23]:
def get_lastest_partition_path(bucket_name, prefix):
    if prefix[-1] != "/": prefix += "/"
    
    s3 = boto3.client('s3')
    paginator = s3.get_paginator('list_objects_v2')
    year_month_day = {}

    for page in paginator.paginate(Bucket=bucket_name, Prefix=prefix):
            for obj in page.get('Contents', []):
                key = obj['Key']
                key = key[len(prefix):]
                parts = key.split('/')
#                 print(parts)
                if len(parts) >= 3:
                    if "=" in parts[0] and "=" in parts[1]:
                        year = parts[0].split('=')[-1]
                        month = parts[1].split('=')[-1]
                        if "=" in parts[2]:
                            day = parts[2].split('=')[-1]
                        else: day = ""
                            
                        if len(day) > 2: continue

                        if year not in year_month_day:
                            year_month_day[year] = {}
                        if month not in year_month_day[year]:
                            year_month_day[year][month] = set()
                        year_month_day[year][month].add(day)

#     print(year_month_day.keys())
    
    most_recent_year = max(year_month_day.keys(), key=int)
    most_recent_month = max(year_month_day[most_recent_year].keys(), key=int)
    if year_month_day[most_recent_year][most_recent_month] != {""}:
        most_recent_day = max(year_month_day[most_recent_year][most_recent_month], key=int)
        final_path = f"{prefix}year={most_recent_year}/month={most_recent_month}/day={most_recent_day}/"
    else:
        final_path = f"{prefix}year={most_recent_year}/month={most_recent_month}/"
        
    return final_path

In [24]:
def create_final_master_file(bucket_cur, gt_parquet_path, proc_parquet_path, pic_parquet_path, prod_path):
    prefix_gtmaster = get_lastest_partition_path(bucket_cur, gt_parquet_path)
    gt_data = gt_parquet_read(bucket_cur, prefix_gtmaster)
    prefix_procmaster = get_lastest_partition_path(bucket_cur, proc_parquet_path)
    proc_data = proc_parquet_read(bucket_cur, prefix_procmaster)
    prefix_picmaster = get_lastest_partition_path(bucket_cur, pic_parquet_path)
    pic_data = pic_parquet_read(bucket_cur, prefix_picmaster)
    
    pg_data = prod_master_read(bucket_cur, prod_path)
    
    pic_data['ID'] = pic_data['ID'].astype(str)
    gt_data['PART_NO'] = gt_data['PART_NO'].astype(str)
    proc_data['PART_NO'] = proc_data['PART_NO'].astype(str)
    pg_data['PART_NO'] = pg_data['PART_NO'].astype(str)
    
    final_master_file_data = pd.merge(pd.merge(pd.merge(gt_data, proc_data, on='PART_NO', how='left'), 
                                 pic_data, left_on='PROCUREMENT_OPERATOR_ID', 
                                 right_on='ID', how='left').drop('ID', axis=1), 
                        pg_data, on='PART_NO', how='left')
    
    return final_master_file_data

# def create_final_master_file(bucket_cur, gt_parquet_path, proc_parquet_path, pic_parquet_path, prod_path):
#     gt_data = gt_parquet_read(bucket_cur, gt_parquet_path)
#     proc_data = proc_parquet_read(bucket_cur, proc_parquet_path)
#     pic_data = pic_parquet_read(bucket_cur, pic_parquet_path)
    
#     pg_data = prod_master_read(bucket_cur, prod_path)
    
#     pic_data['ID'] = pic_data['ID'].astype(str)
#     gt_data['PART_NO'] = gt_data['PART_NO'].astype(str)
#     proc_data['PART_NO'] = proc_data['PART_NO'].astype(str)
#     pg_data['PART_NO'] = pg_data['PART_NO'].astype(str)
    
#     final_master_file_data = pd.merge(pd.merge(pd.merge(gt_data, proc_data, on='PART_NO', how='left'), 
#                                  pic_data, left_on='PROCUREMENT_OPERATOR_ID', 
#                                  right_on='ID', how='left').drop('ID', axis=1), 
#                         pg_data, on='PART_NO', how='left')
    
#     return final_master_file_data

In [25]:
full_master_details = create_final_master_file(bucket_curated, prefix_gtmaster, prefix_proc, prefix_pic_master_path, product_master_path_prefix)

In [26]:
full_master_details.shape, full_master_details['PART_NO'].nunique()

((631326, 8), 631326)

In [27]:
def dashboard_builder(old_forecasting_data_bestfit, final_master_data):
    old_forecasting_data_bestfit_set = old_forecasting_data_bestfit[['part_number', 'model_name','Actual',
               'Forecasted','Accuracy_Pred', 'Confidence_interval', 'Threshold','date']]
    old_forecasting_data_bestfit_set = old_forecasting_data_bestfit_set.rename(columns = {'part_number': 'Part Number', 'model_name': 'Model Name', 
                                                    'Accuracy_Pred': 'New Model Accuracy', 
                                                    'date': 'Date'
                                                   })
    old_forecasting_data_bestfit_set['Lag'] = 'Lag1'
    old_forecasting_data_bestfit_set['Part Number'] = old_forecasting_data_bestfit_set['Part Number'].astype(str)
    
    old_forecasting_data_bestfit_set_master = old_forecasting_data_bestfit_set.merge(final_master_data, left_on='Part Number', 
                                                             right_on='PART_NO', 
                                                             how='left').drop(['PART_NO', 'PROCUREMENT_OPERATOR_ID'], 
                                                                              axis=1).rename(columns={'ICC_seg':'ICC',
                                                                                                      'Vendor Code':'SUPPLIER_NAME',
                                                                                                      'PIC_ABB':'PIC (Abb)'
                                                                                                     })
    
    all_data_sorted_forecasted = old_forecasting_data_bestfit_set_master[['Part Number', 'Model Name', 'ICC','Date', 
                                                                          'Actual', 'Forecasted','New Model Accuracy', 
                                                                          'PRODUCT_GROUP', 'IMPORT_CD', 'Lag', 'SUPPLIER_NAME', 
                                                                          'PIC', 'PIC (Abb)', 'Threshold']]
    
    all_data_sorted_forecasted['Year'] = pd.to_datetime(all_data_sorted_forecasted['Date']).dt.year
    all_data_sorted_forecasted['Month Name'] = pd.to_datetime(all_data_sorted_forecasted['Date']).dt.month_name()
    all_data_sorted_forecasted['Month'] = pd.to_datetime(all_data_sorted_forecasted['Date']).dt.month
    all_data_sorted_forecasted['sort'] = pd.to_datetime(all_data_sorted_forecasted['Date']).dt.year.astype(str) + pd.to_datetime(all_data_sorted_forecasted['Date']).dt.month.astype(str)
    all_data_sorted_forecasted['months'] = pd.to_datetime(all_data_sorted_forecasted['Date']).dt.strftime('%b').astype(str)+pd.to_datetime(all_data_sorted_forecasted['Date']).dt.strftime('%y').astype(str)
    #    rename columns
    col_names_list = """Part Number
    Model Name
    ICC
    Date
    Actual
    Forecasted
    Old Model Accuracy
    New Model Accuracy
    Product Group
    Source Type
    Lag
    Supplier Name
    Person in Charge
    PIC (Abb)
    Threshold
    Year
    Month Name
    Month
    sort
    months""".split('\n')
    col_names_list = list(map(lambda x:x.strip(), col_names_list))
    all_data_sorted_forecasted['Old Model Accuracy']=np.nan
    all_data_sorted_forecasted = all_data_sorted_forecasted[['Part Number', 'Model Name', 'ICC', 'Date', 'Actual', 'Forecasted','Old Model Accuracy',
           'New Model Accuracy', 'PRODUCT_GROUP', 'IMPORT_CD', 'Lag',
           'SUPPLIER_NAME', 'PIC', 'PIC (Abb)', 'Threshold','Year', 'Month Name', 'Month',
           'sort', 'months']]
    all_data_sorted_forecasted.columns = col_names_list
    all_data_sorted_forecasted['Forecast_flag'] = 'Future'
    return all_data_sorted_forecasted

In [28]:
sorted_final_data=dashboard_builder(final_data_compared_bestfit_1,full_master_details)

In [29]:
file_date_forecasting = pd.to_datetime(config.get_value('model_forecasting_date')).strftime("%Y-%m-%d")
filename_forecast_powerbi = f'{config.get_value("filename_forecast_powerbi")}{file_date_forecasting}.xlsx'
file_path_forecast_powerbi = f's3://{bucket_new}/{prefix_forecast_file}{filename_forecast_powerbi}'
sorted_final_data.to_excel(file_path_forecast_powerbi, index=False)

In [30]:
sorted_final_data.columns

Index(['Part Number', 'Model Name', 'ICC', 'Date', 'Actual', 'Forecasted',
       'Old Model Accuracy', 'New Model Accuracy', 'Product Group',
       'Source Type', 'Lag', 'Supplier Name', 'Person in Charge', 'PIC (Abb)',
       'Threshold', 'Year', 'Month Name', 'Month', 'sort', 'months',
       'Forecast_flag'],
      dtype='object')

In [31]:
file_path_forecast_powerbi

's3://s3-dx-tdem-ie-nonprod-curated/curated-bo/curated-bo-demand-forecast/selected-models-forecasted-output/forecast_selected_models_future_months_powerbi_2024-11-03.xlsx'